In [ ]:
#default_exp build_lib

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *
import pprint

## Parse Open API Spec

This library leverages the [OpenAPI Specification](https://github.com/OAI/OpenAPI-Specification) to create a python client for the GitHub API.  The OpenAPI specification contains metadata on all of the endpoints and how to access them properly.  Using this metadata, we can construct a python client dynamically that updates automatically along with the OpenAPI Spec. 

We can download the OpenAPI Spec for GitHub's REST API at the following URL:

In [ ]:
#export
#export
GH_OPENAPI_URL = 'https://github.com/github/rest-api-description/raw/main/descriptions/api.github.com/api.github.com.json?raw=true'
GH_DOC_URL = 'https://docs.github.com/'


In [ ]:
assert urlcheck(GH_OPENAPI_URL), "The URL for the GitHub OpenAPI spec has changed."
assert urlcheck(GH_DOC_URL, "The URL for the GitHub docs has changed.")

Below is an example of the metadata contained for the endpoint `/repos/{owner}/{repo}/git/ref/{ref}`

In [ ]:
js = urljson(GH_OPENAPI_URL)
js['paths']['/repos/{owner}/{repo}/git/ref/{ref}']

{'get': {'summary': 'Get a reference',
  'description': 'Returns a single reference from your Git database. The `:ref` in the URL must be formatted as `heads/<branch name>` for branches and `tags/<tag name>` for tags. If the `:ref` doesn\'t match an existing ref, a `404` is returned.\n\n**Note:** You need to explicitly [request a pull request](https://docs.github.com/rest/reference/pulls#get-a-pull-request) to trigger a test merge commit, which checks the mergeability of pull requests. For more information, see "[Checking mergeability of pull requests](https://docs.github.com/rest/guides/getting-started-with-the-git-database-api#checking-mergeability-of-pull-requests)".',
  'tags': ['git'],
  'operationId': 'git/get-ref',
  'externalDocs': {'description': 'API method documentation',
   'url': 'https://docs.github.com/rest/reference/git#get-a-reference'},
  'parameters': [{'$ref': '#/components/parameters/owner'},
   {'$ref': '#/components/parameters/repo'},
   {'name': 'ref',
    'desc

## Extract Metadata Into A Python Module

In [ ]:
#export
def build_funcs(pre,
                nm='ghapi/metadata.py', 
                url=GH_OPENAPI_URL, 
                docurl=GH_DOC_URL):
    """Build module funcs.py from an Open API spec and optionally filter by a path `pre`"""
    def _get_detls(o):
        data = nested_idx(o, *'requestBody content application/json schema properties'.split()) or {}
        url = o['externalDocs']['url'][len(docurl):]
        return (o['operationId'], o['summary'], url, list(data.keys()))
    
    js = urljson(url)
    paths = {o[len(pre):]:v for o,v in js['paths'].items() if o.startswith(pre)}
    _funcs = [(path, verb) + _get_detls(detls)
              for path,verbs in paths.items() for verb,detls in verbs.items()]
    Path(nm).write_text("funcs = " + pprint.pformat(_funcs, width=360))

For example we will build the module `metadata.py` with the github OpenAPI spec for only paths beginning with `/repos/{owner}/{repo}`:

In [ ]:
build_funcs(pre='/repos/{owner}/{repo}', nm='ghapi/metadata.py')

This python module contains a list of metadata for each endpoint:

In [ ]:
from ghapi.metadata import funcs
L(funcs)

(#298) [('', 'get', 'repos/get', 'Get a repository', 'v3/repos/#get-a-repository', []),('', 'patch', 'repos/update', 'Update a repository', 'v3/repos/#update-a-repository', ['name', 'description', 'homepage', 'private', 'visibility', 'has_issues', 'has_projects', 'has_wiki', 'is_template', 'default_branch', 'allow_squash_merge', 'allow_merge_commit', 'allow_rebase_merge', 'delete_branch_on_merge', 'archived']),('', 'delete', 'repos/delete', 'Delete a repository', 'v3/repos/#delete-a-repository', []),('/actions/artifacts', 'get', 'actions/list-artifacts-for-repo', 'List artifacts for a repository', 'rest/reference/actions#list-artifacts-for-a-repository', []),('/actions/artifacts/{artifact_id}', 'get', 'actions/get-artifact', 'Get an artifact', 'rest/reference/actions#get-an-artifact', []),('/actions/artifacts/{artifact_id}', 'delete', 'actions/delete-artifact', 'Delete an artifact', 'rest/reference/actions#delete-an-artifact', []),('/actions/artifacts/{artifact_id}/{archive_format}', '

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script('build_lib.ipynb')

Converted build_lib.ipynb.
